## Chapter 9 - Miscellaneous Exercises

### TryIt 9.1 - Lag (Non-Ideal Compensator)
Use the following Python and Python Control Systems library to reproduce Figure 9.13

In [1]:
import numpy as np
import control as ct

# Create transfer function with zeros, poles, and gain
Gu = ct.zpk([],[-1,-2,-10],164.6)   # Create plant transfer function
Gc = ct.zpk([-0.111],[-0.01],1)     # Create lag compensator transfer function
Gce = Gu*Gc                         # Cascade the systems by multiplying

# Place open-loop transfer functions into a negative feedback loop
Tu = ct.feedback(Gu,1)              # Place uncompensated system in a unity feedback loop
Tc = ct.feedback(Gce,1)             # Place compensated system in a unity feedback loop

# Plot step response of both uncompensated and compensated systems and compare
ct.step_response(Tu)

